In [1]:
# Cell 0: shared 모듈 임포트용 경로 설정
import sys
import os

# 상위 디렉토리를 sys.path에 추가
sys.path.append(os.path.abspath(".."))

In [2]:
# ✅ Cell 0: GPU 환경 확인
import torch
print("✅ Torch version:", torch.__version__)
print("✅ CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("🚀 CUDA device:", torch.cuda.get_device_name(0))
else:
    print("⚠ GPU not available")

✅ Torch version: 2.6.0+cu124
🔍 CUDA available: True
🚀 Current GPU: Quadro RTX 5000


In [3]:
# ✅ Cell 1: 모든 HDF5 파일 자동 로딩 및 정렬 (shared/data_loader 기반)
from shared.data_loader import HDF5Dataset
import os
import glob

# 디렉토리 경로 설정
input_dir = "/caefs/data/IllustrisTNG/subcube/input"
output_dir = "/caefs/data/IllustrisTNG/subcube/output"

# glob + 정렬을 통해 모든 파일 자동 수집
input_files = sorted(glob.glob(os.path.join(input_dir, "subcubes_stride2_50mpc_parallel_*.h5")))
output_files = sorted(glob.glob(os.path.join(output_dir, "subcubes_stride2_50mpc_parallel_*.h5")))

# 기본 검증 출력
print(f"📁 Found {len(input_files)} input files")
print(f"📁 Found {len(output_files)} output files")

# 데이터셋 생성 및 첫 샘플 확인
dataset = HDF5Dataset(input_files, output_files)
x, y = dataset[0]

print(f"✅ Sample loaded: Input shape = {x.shape}, Output shape = {y.shape}")


TypeError: HDF5Dataset.__init__() missing 1 required positional argument: 'output_files'

In [ ]:
# ✅ Cell 2: U-Net 모델 불러오기 및 GPU 할당
from models.unet3d import UNet3D

torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet3D().to(device)
x = x.unsqueeze(0).to(device)  # (1, 1, D, H, W)
y = y.unsqueeze(0).to(device)

In [ ]:
# ✅ Cell 3: Forward Pass 테스트
model.eval()
with torch.no_grad():
    y_pred = model(x)

print(f"✅ Output shape from UNet3D: {y_pred.shape}")

In [ ]:
# ✅ Cell 4: MSE loss 계산
import torch.nn as nn
criterion = nn.MSELoss()
loss = criterion(y_pred, y)
print(f"🎯 MSE Loss: {loss.item():.6f}")

In [ ]:
# ✅ Cell 5: 간단한 학습 루프 테스트 (3 step만)
from torch.utils.data import DataLoader
import torch.optim as optim

loader = DataLoader(dataset, batch_size=2, shuffle=True)
model = UNet3D().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

model.train()
for i, (xb, yb) in enumerate(loader):
    xb, yb = xb.to(device), yb.to(device)
    optimizer.zero_grad()
    pred = model(xb)
    loss = criterion(pred, yb)
    loss.backward()
    optimizer.step()
    print(f"✅ Step {i}: loss = {loss.item():.6f}")
    if i == 2:
        break

In [ ]:
# ✅ Cell 6: 예측 결과 HDF5 저장
import h5py
import numpy as np

with torch.no_grad():
    pred = model(x).cpu().numpy()
    gt = y.cpu().numpy()

with h5py.File("predicted_sample.h5", "w") as f:
    f.create_dataset("subcubes", data=pred)
    f.create_dataset("ground_truth", data=gt)

print("💾 Saved predicted_sample.h5 with prediction and ground truth.")
